In [1]:
import pandas as pd 
import numpy as np
import yfinance as yf
import datetime as dt

In [2]:
# Download historical data for required stocks
ticker = "AAPL"
ohlcv = yf.download(ticker,dt.date.today()-dt.timedelta(1825),dt.datetime.today())

[*********************100%***********************]  1 of 1 completed


In [3]:
def RSI(DF,n):
    "function to calculate RSI"
    df = DF.copy()
    df['delta']=df['Adj Close'] - df['Adj Close'].shift(1)
    df['gain']=np.where(df['delta']>=0,df['delta'],0)
    df['loss']=np.where(df['delta']<0,abs(df['delta']),0)
    avg_gain = []
    avg_loss = []
    gain = df['gain'].tolist()
    loss = df['loss'].tolist()
    for i in range(len(df)):
        if i < n:
            avg_gain.append(np.NaN)
            avg_loss.append(np.NaN)
        elif i == n:
            avg_gain.append(df['gain'].rolling(n).mean().tolist()[n])
            avg_loss.append(df['loss'].rolling(n).mean().tolist()[n])
        elif i > n:
            avg_gain.append(((n-1)*avg_gain[i-1] + gain[i])/n) # I think this is also known as smooth averaging 
            avg_loss.append(((n-1)*avg_loss[i-1] + loss[i])/n)
    df['avg_gain']=np.array(avg_gain)
    df['avg_loss']=np.array(avg_loss)
    df['RS'] = df['avg_gain']/df['avg_loss']
    df['RSI'] = 100 - (100/(1+df['RS']))
    return df['RSI']

In [4]:
# Calculating RSI without using loop
def rsi(df, n):
    "function to calculate RSI"
    delta = df["Adj Close"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    return 100 - 100 / (1+rs)

In [5]:
df = rsi(ohlcv, 10)

In [7]:
df.head(50)

Date
2016-02-08          NaN
2016-02-09          NaN
2016-02-10          NaN
2016-02-11          NaN
2016-02-12          NaN
2016-02-16          NaN
2016-02-17          NaN
2016-02-18          NaN
2016-02-19          NaN
2016-02-22    59.262308
2016-02-23    44.449804
2016-02-24    52.804173
2016-02-25    56.196406
2016-02-26    56.969450
2016-02-29    55.392154
2016-03-01    70.875318
2016-03-02    71.498769
2016-03-03    73.616555
2016-03-04    77.344834
2016-03-07    69.221557
2016-03-08    63.790993
2016-03-09    64.122788
2016-03-10    64.322474
2016-03-11    68.524647
2016-03-14    69.468130
2016-03-15    75.792669
2016-03-16    79.018532
2016-03-17    77.626966
2016-03-18    77.928773
2016-03-21    77.832752
2016-03-22    80.030412
2016-03-23    74.140959
2016-03-24    69.739469
2016-03-28    65.290415
2016-03-29    74.553566
2016-03-30    79.170376
2016-03-31    74.653284
2016-04-01    77.167327
2016-04-04    79.673483
2016-04-05    69.891055
2016-04-06    73.082474
2016-04-07 

In [9]:
df = RSI(ohlcv, 10)

In [10]:
df.head(50)

Date
2016-01-25          NaN
2016-01-26          NaN
2016-01-27          NaN
2016-01-28          NaN
2016-01-29          NaN
2016-02-01          NaN
2016-02-02          NaN
2016-02-03          NaN
2016-02-04          NaN
2016-02-05          NaN
2016-02-08    40.278711
2016-02-09    40.234096
2016-02-10    38.525819
2016-02-11    37.138822
2016-02-12    38.392700
2016-02-16    48.768088
2016-02-17    53.615663
2016-02-18    47.358356
2016-02-19    46.642948
2016-02-22    49.856499
2016-02-23    42.450368
2016-02-24    47.978489
2016-02-25    50.453953
2016-02-26    51.042127
2016-02-29    50.073317
2016-03-01    63.507378
2016-03-02    64.121930
2016-03-03    66.273238
2016-03-04    70.262281
2016-03-07    63.920490
2016-03-08    59.521868
2016-03-09    59.850845
2016-03-10    60.051160
2016-03-11    64.359590
2016-03-14    65.349931
2016-03-15    72.160549
2016-03-16    75.736095
2016-03-17    74.437039
2016-03-18    74.776372
2016-03-21    74.684789
2016-03-22    77.203625
2016-03-23 